<a href="https://colab.research.google.com/github/pipuf/ml_dev_cert/blob/main/15_1_2_PRACTICE_MLP_for_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cifar-10 classification

Cifar-10 is a dataset having 60000 color images with 32x32x3 pixels each one.

Each image belongs to one out of 10 possible categories. Having exactly 6000 images per class.

We have 50000 images for training and 10000 for testing.

Let's use MLP models to solve this!

In [ ]:
from keras.datasets import cifar10

## Load data

Keras will download it automatically.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape: {}'.format(x_train.shape))
print('x_test shape: {}'.format(x_test.shape))

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


## [Complete] Plot some images for each class

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler


x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))


# Initialize the scalers
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Apply MinMaxScaler
x_train_minmax = min_max_scaler.fit_transform(x_train_flat)
x_test_minmax = min_max_scaler.transform(x_test_flat)

# Apply StandardScaler
x_train_standard = standard_scaler.fit_transform(x_train_flat)
x_test_standard = standard_scaler.transform(x_test_flat)

## Feature engineering

- Convert your data shape so you can use them with your MLP
- Scale your images
  - Try using MinMax and StandardScaler, compare both methods

In [ ]:
import numpy as np

# Example data
x_train = np.random.rand(50000, 32, 32, 3)
x_test = np.random.rand(10000, 32, 32, 3)

# Reshape the data
x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

print(f"Reshaped x_train: {x_train_flat.shape}")
print(f"Reshaped x_test: {x_test_flat.shape}")

Reshaped x_train: (50000, 3072)
Reshaped x_test: (10000, 3072)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Define MLP model with various configurations
def create_mlp_model(input_shape, layers_config, activation, weight_init, dropout_rate, l2_reg):
    model = Sequential()
    for i, units in enumerate(layers_config):
        if i == 0:
            model.add(Dense(units, activation=activation, kernel_initializer=weight_init, kernel_regularizer=l2(l2_reg), input_shape=input_shape))
        else:
            model.add(Dense(units, activation=activation, kernel_initializer=weight_init, kernel_regularizer=l2(l2_reg)))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Training and evaluation function
def train_and_evaluate(x_train, y_train, x_test, y_test, layers_config, activation, weight_init, dropout_rate, l2_reg):
    input_shape = (x_train.shape[1],)
    model = create_mlp_model(input_shape, layers_config, activation, weight_init, dropout_rate, l2_reg)
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=1)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    return test_acc

# Different configurations to test
configs = [
    {'layers': [128, 64], 'activation': 'relu', 'weight_init': 'he_normal', 'dropout_rate': 0.5, 'l2_reg': 1e-4},
    {'layers': [256, 128, 64], 'activation': 'tanh', 'weight_init': 'glorot_uniform', 'dropout_rate': 0.3, 'l2_reg': 1e-3},
    {'layers': [512, 256], 'activation': 'relu', 'weight_init': 'he_uniform', 'dropout_rate': 0.4, 'l2_reg': 1e-2},
    {'layers': [128, 128, 128], 'activation': 'sigmoid', 'weight_init': 'glorot_normal', 'dropout_rate': 0.2, 'l2_reg': 1e-5}
]

# Evaluate all configurations on both scaling methods
results = []
for config in configs:
    acc_minmax = train_and_evaluate(x_train_minmax, y_train, x_test_minmax, y_test, config)
    acc_standard = train_and_evaluate(x_train_standard, y_train, x_test_standard, y_test, config)
    results.append((config, acc_minmax, acc_standard))

# Print results
for config, acc_minmax, acc_standard in results:
    print(f"Config: {config}")
    print(f"MinMax Scaler Accuracy: {acc_minmax}")
    print(f"Standard Scaler Accuracy: {acc_standard}\n")

TypeError: train_and_evaluate() missing 4 required positional arguments: 'activation', 'weight_init', 'dropout_rate', and 'l2_reg'

## Model training

Use keras tuner to find the best hyperparameters


## Model evaluation

Use different evaluation metrics and your testing dataset to report how good your model is.

Think using scikit-learn `classification_report` and confussion matrix to generate good quality reports.
